In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from utils.preprocessingCheckpoint import getPreprocessingCheckpoint

In [3]:
trainData, validationData = getPreprocessingCheckpoint()

In [4]:
features = trainData["features"]
targets = trainData["targets"]

In [5]:
len(features), features[0].shape

(29364, (420, 5))

In [6]:
len(targets), np.unique(targets)

(29364, array([0, 1, 2]))

In [7]:
encoder = {
    "0": np.array([[1.0, 0.0, 0.0]]),
    "1": np.array([[0.0, 1.0, 0.0]]),
    "2": np.array([[0.0, 0.0, 1.0]]),
}

In [8]:
encodedTargets = [ encoder[str(label)] for label in targets ]

In [9]:
len(encodedTargets), np.unique(encodedTargets, axis=0)

(29364,
 array([[[0., 0., 1.]],
 
        [[0., 1., 0.]],
 
        [[1., 0., 0.]]]))

In [10]:
x_train, x_test, y_train, y_test = train_test_split(
    features, encodedTargets, test_size=0.4, stratify=targets
)

In [11]:
len(features), len(x_train), len(x_test)

(29364, 17618, 11746)

In [12]:
len(targets), len(y_train), len(y_test)

(29364, 17618, 11746)

In [13]:
len(x_train), x_train[0], x_train[0].shape, x_train[0][0], x_train[0][0].shape, x_train[0][0][0], type(x_train[0][0][0])

(17618,
 array([[-0.10695352, -0.44400167,  1.35172209, -0.57731491,  0.83459091],
        [ 0.25379129,  0.35595293,  1.2603215 , -0.48374891,  1.04629756],
        [ 3.93762115, -1.44287488,  1.33043028, -1.45774055,  1.2076112 ],
        ...,
        [-0.18663607, -0.39838875,  0.85330764, -0.0679976 ,  1.65713902],
        [-0.06651999,  0.3102997 ,  0.51949405, -0.7419467 ,  1.39165561],
        [ 0.55202946, -0.84420596,  0.85992282, -0.40497342,  1.00809348]]),
 (420, 5),
 array([-0.10695352, -0.44400167,  1.35172209, -0.57731491,  0.83459091]),
 (5,),
 -0.10695351991082178,
 numpy.float64)

In [14]:
len(y_train), y_train[0], y_train[0].shape, y_train[0][0], y_train[0][0].shape, y_train[0][0][0], type(y_train[0][0][0])

(17618,
 array([[1., 0., 0.]]),
 (1, 3),
 array([1., 0., 0.]),
 (3,),
 1.0,
 numpy.float64)

In [15]:
y_train[0].shape, x_train[0].shape

((1, 3), (420, 5))

In [16]:
from reservoirpy.nodes import Reservoir, Ridge, Input
from reservoirpy import set_seed, verbosity

In [17]:
set_seed(42)
verbosity(0)

0

In [18]:
source = Input()
reservoir = Reservoir(4, sr=0.9, lr=0.1)
readout = Ridge(ridge=1e-6)

model = source >> reservoir >> readout

In [19]:
states_train = []
for x in x_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])

In [20]:
readout.fit(states_train, y_train)

'Ridge-0': Ridge(ridge=1e-06, input_bias=True, in=4, out=3)

In [21]:
y_pred = []
for x in x_test:
    states = reservoir.run(x, reset=True)
    y = readout.run(states[-1, np.newaxis])
    y_pred.append(y)

In [22]:
y_pred_class = [np.argmax(y_p) for y_p in y_pred]
y_test_class = [np.argmax(y_t) for y_t in y_test]

score = accuracy_score(y_test_class, y_pred_class)

print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  65.673 %
